In [1]:
import numpy as np 
import astropy.constants as c
import os
import csv
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import curve_fit
import math
#list all the constants used
G=c.G.value
M_sun=c.M_sun.value
M_earth=c.M_earth.value
R_sun=c.R_sun.value
R_earth=c.R_earth.value
hour=3600 #s
day=3600*24
e=0.1
t_LC=1765.5 #s


def delta(Rp, Rs):
    delta = (Rp*R_earth / (Rs*R_sun))**2*1e6
    return delta

def SNR(delta,n,sigma_s):
    SNR=delta*n**0.5/sigma_s
    return SNR

def sigma_s(t, sigma_LC, cdppindex): #t/hour
    t_LC=1765.5/hour 

    return sigma_LC*(t/t_LC)**cdppindex

def t_dur(P,Rs,e,a): #hour
    t_dur = P*day*Rs*R_sun*(1-e**2)/(np.pi*a)/hour
    return t_dur

def a(Ms,P):
    a=((G*Ms*M_sun*(P*day)**2)/(4*np.pi**2))**(1/3)
    return a 

def n(t_obs,P): # the unite of t_obs and P is day
    n=t_obs/P
    return n

def f_n(t_obs,P):
    if t_obs <=2*P:
        return 0
    if t_obs> 2*P and t_obs< 3*P:
        return (t_obs/P-2)
    if t_obs>=3*P:
        return 1
    
def f_eff(SNR):
    if SNR <=6:
        return 0
    if SNR >6 and SNR<=12:
        return (SNR-6)/6
    if SNR>12:
        return 1
    
def f_geo(Rp,Rs,a,e):
    f_geo = (Rp*R_earth+Rs*R_sun)/(a*(1-e**2))
    return f_geo

In [2]:
stellar_table=pd.read_csv('data/kepler_stellar_q17_add_dispersions_reduction_reduction_2020.csv',sep=',')
koi_table=pd.read_csv('data/kepler_koi_q17_add_dispersions_reduction_reduction_new.csv',sep=',')

def N_s(Rp,P,i):
    os.system('mkdir result/N_tot_2020_new/')
    N_s_temp=0
    with open('result/N_tot_2020_new/'+str(i)+'.csv','w') as f:
        s=['SNR',
           'fn',
           'feff',]
        writer=csv.DictWriter(f,fieldnames=s)
        writer.writeheader()        
        for i in range(0,len(stellar_table)):
            a_temp=a(stellar_table['mass'][i],P)
            tdur_temp=t_dur(P,stellar_table['rad_b2018'][i],e,a_temp)
            delta_temp=delta(Rp,stellar_table['rad_b2018'][i])
            n_temp=n(stellar_table['t_obs'][i],P)
            x=[3,6,12]
            y=[stellar_table['cdpp3'][i],stellar_table['cdpp6'][i],stellar_table['cdpp12'][i]]
            popt, pcov = curve_fit(sigma_s, x, y)
            sigma_LC=popt[0]
            cdppindex=popt[1]
            sigma_s_temp=sigma_s(tdur_temp,sigma_LC,cdppindex)
            SNR_temp=SNR(delta_temp,n_temp,sigma_s_temp)
            fn_temp=f_n(stellar_table['t_obs'][i],P)
            feff_temp=f_eff(SNR_temp)
            writer.writerow({'SNR':SNR_temp,
                             'fn':fn_temp,
                             'feff':feff_temp,})                             
            N_s_temp+=fn_temp*feff_temp
    return round(N_s_temp)

def N_s2(Rp,P):
#    os.system('mkdir result/N_s/')
    N_s_temp=0
#    with open('result/N_s/'+str(kepid)+'.csv','w') as f:
#        s=['SNR',
#           'fn',
#           'feff',]
#        writer=csv.DictWriter(f,fieldnames=s)
#        writer.writeheader()        
    for i in range(0,len(stellar_table)):
        a_temp=a(stellar_table['mass'][i],P)
        tdur_temp=t_dur(P,stellar_table['rad_b2018'][i],e,a_temp)
        delta_temp=delta(Rp,stellar_table['rad_b2018'][i])
        n_temp=n(stellar_table['t_obs'][i],P)
        x=[3,6,12]
        y=[stellar_table['cdpp3'][i],stellar_table['cdpp6'][i],stellar_table['cdpp12'][i]]
        popt, pcov = curve_fit(sigma_s, x, y)
        sigma_LC=popt[0]
        cdppindex=popt[1]
        sigma_s_temp=sigma_s(tdur_temp,sigma_LC,cdppindex)
        SNR_temp=SNR(delta_temp,n_temp,sigma_s_temp)
        fn_temp=f_n(stellar_table['t_obs'][i],P)
        feff_temp=f_eff(SNR_temp)
#        writer.writerow({'SNR':SNR_temp,
#                         'fn':fn_temp,
#                         'feff':feff_temp,})                             
        N_s_temp+=fn_temp*feff_temp
    return round(N_s_temp)

def f_occurrence_dot(Rp,P,Ms,Rs,e,i):
    N_s_temp=N_s(Rp,P,i)
    a_temp=a(Ms,P)
    fgeo_temp=f_geo(Rp,Rs,a_temp,e)
    f_occurrence_temp=1./(fgeo_temp*N_s_temp)
    return f_occurrence_temp
    

def f_occurrence_cell_nofp(Rp_low,Rp_high,P_low,P_high,f_occurrence_everyplanet): # the log base of Rp and P
    
    count=0
    for i in range(0,len(koi_table)):
        if koi_table['koi_prad'][i]>Rp_low and koi_table['koi_prad'][i]<=Rp_high and koi_table['koi_period'][i]>P_low and koi_table['koi_period'][i]<=P_high\
        and koi_table['koi_disposition'][f_id[j]]!='FALSE POSITIVE':
            count+=1
    
    if count>0:
        f_occurrence_temp=0
        for i in range(0,len(koi_table)):
            if koi_table['koi_prad'][i]>Rp_low and koi_table['koi_prad'][i]<=Rp_high and koi_table['koi_period'][i]>P_low and koi_table['koi_period'][i]<=P_high\
            and koi_table['koi_disposition'][f_id[j]]!='FALSE POSITIVE':
#                N_s_temp=N_s(koi_table['koi_prad'][i],koi_table['koi_period'][i])
#                a_temp=a(koi_table['mass'][i],koi_table['koi_period'][i])
#                fgeo_temp=f_geo(koi_table['koi_prad'][i],koi_table['rad_b2018'][i],a_temp,e)
                if f_occurrence_everyplanet[i]!=np.inf:
                    f_occurrence_temp+=float(f_occurrence_everyplanet[i])
        
        temp=1.0*count/f_occurrence_temp
        if f_occurrence_temp>=1:
            return count,f_occurrence_temp,0
        else:
            deviation_N_s=temp*f_occurrence_temp*(1-f_occurrence_temp)
            std_N_s=deviation_N_s**0.5
            std_cell=std_N_s/temp
            return count,f_occurrence_temp,std_cell
    
    else:
        base_period=5
        base_radius=2
        Rp_center=base_radius**((math.log(Rp_low,base_radius)+math.log(Rp_high,base_radius))/2)
        P_center=base_period**((math.log(P_low,base_period)+math.log(P_high,base_period))/2)
        N_s_temp=N_s2(Rp_center,P_center)
        fgeo_temp=0.1
        if N_s_temp==0:
            f_occurrence_temp=0
        else:
            f_occurrence_temp=1./(fgeo_temp*N_s_temp)
            
        return count,f_occurrence_temp,0


In [3]:
for i in range(2316,2318):
    N_s(koi_table['koi_prad'][i],koi_table['koi_period'][i],i)
    print(i)

2316


KeyboardInterrupt: 

1064.2680956
